In [2]:
import tensorflow as tf
print("Current GPU device:\n ", tf.test.gpu_device_name(), "\n")

from tensorflow.python.client import device_lib
print("Current local devices: \n", device_lib.list_local_devices(), "\n")

Current GPU device:
  /device:GPU:0 

Current local devices: 
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1015348113683966996
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12135836880824869137
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13104639511237790959
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2952772485579563763
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
] 



In [3]:
print("RAM info: ")
!cat /proc/meminfo

RAM info: 
MemTotal:       13335276 kB
MemFree:         4670780 kB
MemAvailable:   12068460 kB
Buffers:          167688 kB
Cached:          7187744 kB
SwapCached:            0 kB
Active:          1431932 kB
Inactive:        6676140 kB
Active(anon):     688548 kB
Inactive(anon):     2392 kB
Active(file):     743384 kB
Inactive(file):  6673748 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               816 kB
Writeback:             0 kB
AnonPages:        752780 kB
Mapped:           441092 kB
Shmem:              2948 kB
Slab:             346980 kB
SReclaimable:     304352 kB
SUnreclaim:        42628 kB
KernelStack:        4176 kB
PageTables:         8120 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667636 kB
Committed_AS:    2564812 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemH

In [4]:
print("CPU info: ")
!cat /proc/cpuinfo

CPU info: 
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
ve

## Install XGBoost for GPU

In [5]:
!pip3 install xgboost

In [6]:
import xgboost as xgb
print("XGBoost version: ", xgb.__version__)

import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
import time

# Fetch dataset using sklearn
cov = fetch_covtype()
X = cov.data
y = cov.target

# Create 0.75/0.25 train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75,
                                                    random_state=42)

# Specify sufficient boosting iterations to reach a minimum
num_round = 1000 #3000

# Leave most parameters as default
param = {'objective': 'multi:softmax', # Specify multiclass classification
         'num_class': 8, # Number of possible output classes
         'tree_method': 'gpu_hist' # Use GPU accelerated algorithm
         }

# Convert input data from numpy to XGBoost format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

gpu_res = {} # Store accuracy result
tmp = time.time()
# Train model
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_res, verbose_eval=200)
print("GPU Training Time: %s seconds" % (str(time.time() - tmp)))

# Repeat for CPU algorithm
tmp = time.time()
param['tree_method'] = 'hist'
cpu_res = {}
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_res, verbose_eval=200)
print("CPU Training Time: %s seconds" % (str(time.time() - tmp)))

XGBoost version:  0.82
[0]	test-merror:0.254804
[200]	test-merror:0.091847
[400]	test-merror:0.062987
[600]	test-merror:0.050601
[800]	test-merror:0.043972
[999]	test-merror:0.040571
GPU Training Time: 472.7204933166504 seconds
[0]	test-merror:0.254831
[200]	test-merror:0.090814
[400]	test-merror:0.061809
[600]	test-merror:0.0495
[800]	test-merror:0.043607
[999]	test-merror:0.039538
CPU Training Time: 2021.3367593288422 seconds


### 20190309: numround=1000: GPU= 473s/7min53s,  CPU= 2021s/33min41s. GPU is 4 times faster than CPU.

In [7]:
print(X.shape)
print(y.shape)

(581012, 54)
(581012,)


In [11]:
len(gpu_res["test"]["merror"])

1000

## Install from source, but the default version was still shown (archive - might be usefull)

In [0]:
!pip uninstall --yes xgboost

Skipping xgboost as it is not installed.


In [0]:
#!pip install -q xgboost==0.82

!rm -rf *
!git clone --recursive https://github.com/dmlc/xgboost

Cloning into 'xgboost'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 27389 (delta 2), reused 3 (delta 1), pack-reused 27383
Receiving objects: 100% (27389/27389), 10.35 MiB | 24.75 MiB/s, done.
Resolving deltas: 100% (16126/16126), done.
Submodule 'cub' (https://github.com/NVlabs/cub) registered for path 'cub'
Submodule 'dmlc-core' (https://github.com/dmlc/dmlc-core) registered for path 'dmlc-core'
Submodule 'rabit' (https://github.com/dmlc/rabit) registered for path 'rabit'
Cloning into '/content/xgboost/cub'...
remote: Enumerating objects: 32671, done.        
remote: Total 32671 (delta 0), reused 0 (delta 0), pack-reused 32671        
Receiving objects: 100% (32671/32671), 16.53 MiB | 1.33 MiB/s, done.
Resolving deltas: 100% (28644/28644), done.
Cloning into '/content/xgboost/dmlc-core'...
remote: Enumerating objects: 9, done.        
remote: Counting objects: 100% (9/9), done.       

In [0]:
%cd xgboost
!mkdir build
%cd build
!cmake .. -DUSE_CUDA=ON
!make -j4


/content/xgboost/xgboost
/content/xgboost/xgboost/build
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Setting build type to 'Release' as none was specified.
-- Performing Test XGBOOST_MM_PREFETCH_PRESENT
-- Performing Test XGBOOST_MM_PREFETCH_PRESENT - Success
-- Performing Test XGBOOST_BUILTIN_PRE

In [0]:
%cd ../python-package
!python3 setup.py install

/content/xgboost/xgboost/python-package
Install libxgboost from: ['../lib/libxgboost.so']
running install
running bdist_egg
running egg_info
creating xgboost.egg-info
writing xgboost.egg-info/PKG-INFO
writing dependency_links to xgboost.egg-info/dependency_links.txt
writing requirements to xgboost.egg-info/requires.txt
writing top-level names to xgboost.egg-info/top_level.txt
writing manifest file 'xgboost.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'xgboost.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/xgboost
copying xgboost/sklearn.py -> build/lib/xgboost
copying xgboost/training.py -> build/lib/xgboost
copying xgboost/core.py -> build/lib/xgboost
copying xgboost/callback.py -> build/lib/xgboost
copying xgboost/rabit.py -> build/lib/xgboost
copying xgboost/libpath.py -> build/lib/xgboost
copying xgboost/compat.py -> build/l

In [0]:
!export PYTHONPATH=/content/xgboost/python-package


In [0]:
import xgboost as xgb
xgb.__version__


'0.7.post4'